In [1]:
# Required imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
from dotenv import load_dotenv
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

load_dotenv()
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

In [2]:
FILE_NAME = 'feature_frame.csv'
S3_DIR = 'groceries/box_builder_dataset/'
BUCKET_NAME = 'zrive-ds-data'
S3_PATH = "s3://" + BUCKET_NAME + S3_DIR + FILE_NAME
LOCAL_DATA_PATH = 'data/'
LOCAL_FILE_PATH = LOCAL_DATA_PATH + FILE_NAME

def get_data_from_s3():
    """Download data from S3 if not exists locally."""
    os.makedirs(LOCAL_DATA_PATH, exist_ok=True)
    
    if os.path.exists(LOCAL_FILE_PATH):
        print(f"File already exists at {LOCAL_FILE_PATH}")
        return
    
    try:
        s3 = boto3.client('s3',
                         aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
        
        key = S3_DIR + FILE_NAME
        
        print(f"Downloading from {S3_PATH}")
        s3.download_file(BUCKET_NAME, key, LOCAL_FILE_PATH)
        print(f"File downloaded successfully to {LOCAL_FILE_PATH}")
        
    except Exception as e:
        print(f"Error downloading file: {e}")
        raise

# Download data if needed
get_data_from_s3()

# Load the dataset
df = pd.read_csv(LOCAL_FILE_PATH)


File already exists at data/feature_frame.csv


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880549 entries, 0 to 2880548
Data columns (total 27 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   variant_id                        int64  
 1   product_type                      object 
 2   order_id                          int64  
 3   user_id                           int64  
 4   created_at                        object 
 5   order_date                        object 
 6   user_order_seq                    int64  
 7   outcome                           float64
 8   ordered_before                    float64
 9   abandoned_before                  float64
 10  active_snoozed                    float64
 11  set_as_regular                    float64
 12  normalised_price                  float64
 13  discount_pct                      float64
 14  vendor                            object 
 15  global_popularity                 float64
 16  count_adults                      fl

In [4]:
df.head()

,variant_id,product_type,order_id,user_id,created_at,order_date,user_order_seq,outcome,ordered_before,abandoned_before,...,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
0,33826472919172,ricepastapulses,2807985930372,3482464092292,2020-10-05 16:46:19,2020-10-05 00:00:00,3,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
1,33826472919172,ricepastapulses,2808027644036,3466586718340,2020-10-05 17:59:51,2020-10-05 00:00:00,2,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
2,33826472919172,ricepastapulses,2808099078276,3481384026244,2020-10-05 20:08:53,2020-10-05 00:00:00,4,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
3,33826472919172,ricepastapulses,2808393957508,3291363377284,2020-10-06 08:57:59,2020-10-06 00:00:00,2,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618
4,33826472919172,ricepastapulses,2808429314180,3537167515780,2020-10-06 10:37:05,2020-10-06 00:00:00,3,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.27618


In [5]:
df['outcome'].value_counts(normalize=True)

outcome
0.0    0.988463
1.0    0.011537
Name: proportion, dtype: float64

In [6]:
df['days_since_purchase_product_type'].value_counts()

days_since_purchase_product_type
30.0     2284388
14.0       16494
21.0       14917
29.0       14904
20.0       14037
          ...   
148.0        116
134.0        100
117.0         58
128.0         58
145.0         44
Name: count, Length: 141, dtype: int64

## Filtering Data

Only those purchases with more than 5 products bought

In [7]:
# Paso 1: Agrupar por order_id y contar los productos con outcome=1
items_comprados_por_orden = df[df['outcome'] == 1.0].groupby('order_id').size()

# Paso 2: Crear un DataFrame con esta información
items_comprados_df = items_comprados_por_orden.reset_index(name='items_comprados')

# Paso 3: Hacer un merge con el DataFrame original
df = df.merge(items_comprados_df, on='order_id', how='left')

# Para órdenes que no tienen ningún producto comprado, reemplazar NaN con 0
df['items_comprados'] = df['items_comprados'].fillna(0).astype(int)

In [8]:
valuable_orders_df = df[df['items_comprados'] > 5]
valuable_orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2023050 entries, 0 to 2880547
Data columns (total 28 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   variant_id                        int64  
 1   product_type                      object 
 2   order_id                          int64  
 3   user_id                           int64  
 4   created_at                        object 
 5   order_date                        object 
 6   user_order_seq                    int64  
 7   outcome                           float64
 8   ordered_before                    float64
 9   abandoned_before                  float64
 10  active_snoozed                    float64
 11  set_as_regular                    float64
 12  normalised_price                  float64
 13  discount_pct                      float64
 14  vendor                            object 
 15  global_popularity                 float64
 16  count_adults                      float64

## Building linear model

In [9]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from datetime import datetime
import sys
import re

### Prepare data

Frecuency encoder to product_type feature

In [16]:
def freq_encoder(df: pd.DataFrame, column: str) -> pd.DataFrame:
    freq = df[column].value_counts(normalize=True)
    df[column] = df[column].map(freq)
    return df

valuable_orders_df = freq_encoder(valuable_orders_df, 'product_type')

Process date features.
```
{'date': '2023-10-01',
 'year': '2023',
 'year_s': '23',
 'month_num': '10',
 'dom': '01',
 'doy': '274',
 'woy': '39',
 'dow_num': 7,
 'is_weekend': True}
```

In [11]:
def process_date(input_str: str) -> dict:

    date_str = input_str.split(' ')[0]

    # Validate date string input
    regex = re.compile(r'\d{4}-\d{2}-\d{2}')
    if not re.match(regex, input_str):
        print("Invalid date format")
        sys.exit(1)

    # Process date features
    my_date = datetime.strptime(date_str, '%Y-%m-%d').date()
    date_feats = {}

    date_feats['date'] = date_str
    date_feats['year'] = my_date.strftime('%Y')
    date_feats['year_s'] = my_date.strftime('%y')
    date_feats['month_num'] = my_date.strftime('%m')
    date_feats['dom'] = my_date.strftime('%d')
    date_feats['doy'] = my_date.strftime('%j')
    date_feats['woy'] = my_date.strftime('%W')

    # Fixing day of week to start on Mon (1), end on Sun (7)
    dow = my_date.strftime('%w')
    if dow == '0': dow = 7
    date_feats['dow_num'] = dow

    if int(dow) > 5:
        date_feats['is_weekend'] = True
    else:
        date_feats['is_weekend'] = False

    return date_feats

In [12]:
def process_dates(df: pd.DataFrame) -> pd.DataFrame:
    date_feats = df['order_date'].apply(process_date)
    date_feats_df = pd.DataFrame(list(date_feats))

    # Merge with original DataFrame
    df = pd.concat([df, date_feats_df], axis=1)

    # Drop original order_date column
    df.drop(columns=['order_date'], inplace=True)

    return df

date_df = process_dates(valuable_orders_df)
date_df.head()

,variant_id,product_type,order_id,user_id,created_at,user_order_seq,outcome,ordered_before,abandoned_before,active_snoozed,...,items_comprados,date,year,year_s,month_num,dom,doy,woy,dow_num,is_weekend
0,3.382647e+13,ricepastapulses,2.807986e+12,3.482464e+12,2020-10-05 16:46:19,3.0,0.0,0.0,0.0,0.0,...,9.0,2020-10-05,2020,20,10,05,279,40,1,False
1,3.382647e+13,ricepastapulses,2.808028e+12,3.466587e+12,2020-10-05 17:59:51,2.0,0.0,0.0,0.0,0.0,...,6.0,2020-10-05,2020,20,10,05,279,40,1,False
2,3.382647e+13,ricepastapulses,2.808099e+12,3.481384e+12,2020-10-05 20:08:53,4.0,0.0,0.0,0.0,0.0,...,9.0,2020-10-05,2020,20,10,05,279,40,1,False
3,3.382647e+13,ricepastapulses,2.808394e+12,3.291363e+12,2020-10-06 08:57:59,2.0,0.0,0.0,0.0,0.0,...,13.0,2020-10-06,2020,20,10,06,280,40,2,False
5,3.382647e+13,ricepastapulses,2.808435e+12,3.479091e+12,2020-10-06 10:50:23,3.0,0.0,0.0,0.0,0.0,...,7.0,2020-10-06,2020,20,10,06,280,40,2,False


In [17]:
X = valuable_orders_df.drop(columns=['outcome', 'order_id', 'user_id', 'variant_id', 'items_comprados', 'created_at', 'order_date'])
y = valuable_orders_df['outcome']

X

,product_type,user_order_seq,ordered_before,abandoned_before,active_snoozed,set_as_regular,normalised_price,discount_pct,vendor,global_popularity,...,count_children,count_babies,count_pets,people_ex_baby,days_since_purchase_variant_id,avg_days_to_buy_variant_id,std_days_to_buy_variant_id,days_since_purchase_product_type,avg_days_to_buy_product_type,std_days_to_buy_product_type
0,0.044599,3,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.000000,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
1,0.044599,2,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.000000,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
2,0.044599,4,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.000000,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
3,0.044599,2,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.038462,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
5,0.044599,3,0.0,0.0,0.0,0.0,0.081052,0.053512,clearspring,0.038462,...,0.0,0.0,0.0,2.0,33.0,42.0,31.134053,30.0,30.0,24.276180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2880539,0.007216,4,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,...,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
2880541,0.007216,2,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,...,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
2880544,0.007216,3,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,...,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
2880546,0.007216,7,0.0,0.0,0.0,0.0,0.417186,0.114360,colief,0.000000,...,0.0,0.0,0.0,2.0,33.0,34.0,27.693045,30.0,34.0,27.451392
